In [1]:
# Imports and Helper Functions
# data Analysis
import pandas as pd
import numpy as np
import random as rng

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr" -- povrat

In [ ]:

pd.options.display.float_format = '{:,.2f}'.format
data = pd.read_csv("data.csv")
fighters_data = pd.read_csv("fighters.csv")
categories_data = pd.read_csv("categories.csv")
fighter_mean = pd.read_csv("fighter_means.csv")

In [ ]:
#data.head()
fighters_data.head()
categories_data.head()
data.tail()

In [ ]:
data.dtypes
data.shape
data.info()

In [ ]:
data.describe(include=['object']) 
#deskriptivan statiskita za brojčane varijable
data.describe()

In [ ]:
# Drop rows with NaN values in column 'A'
#data.dropna(subset=['winner'], inplace=True)
#data["winner"].unique()

data = data[(data["winner"] == "red") | (data["winner"] == "blue")]

In [5]:
fighters=data["B_Name"].unique()
np.savetxt('fighters.csv', fighters, delimiter=',', fmt='%s', header='Fighters', comments='')
#fighters

In [35]:
#data.fillna(value=0, inplace=True)
data.loc[:, :] = data.fillna(value=0)

In [32]:
dropdata = data.drop(['B_Location', 'B_HomeTown', 'B_ID','B_Name', 'R_Location', 'R_HomeTown','R_ID','R_Name','winby','Date'], axis=1)
dropdata.rename(columns={'BPrev':'B__Prev',
                        'RPrev':'R__Prev',
                        'B_Age':'B__Age',
                        'B_Height':'B__Height',
                        'B_Weight':'B__Weight',
                        'R_Age':'R__Age',
                        'R_Height':'R__Height',
                        'R_Weight':'R__Weight',
                        'BStreak':'B__Streak',
                        'RStreak': 'R__Streak'}, inplace=True)

In [40]:
dropdata = data.drop(['Event_ID', 'Fight_ID', 'B_Location', 'B_HomeTown', 'B_ID', 'R_Location', 'R_HomeTown','R_ID','winby','Date', 'winner'], axis=1)
dropdata.rename(columns={'BPrev':'B__Prev',
                        'RPrev':'R__Prev',
                        'B_Age':'B__Age',
                        'B_Height':'B__Height',
                        'B_Weight':'B__Weight',
                        'R_Age':'R__Age',
                        'R_Height':'R__Height',
                        'R_Weight':'R__Weight',
                        'BStreak':'B__Streak',
                        'RStreak': 'R__Streak',
                        'B_Name': 'B__Name',
                        'R_Name': 'R__Name'}, inplace=True)

combined_all = pd.DataFrame()
for i, name in enumerate(fighters_data['Fighters']):
    conor = dropdata.loc[ (dropdata['B__Name']  == name)]
    filtered_columns = conor.filter(regex='^R', axis=1)
    conorB = conor.drop(filtered_columns.columns, axis=1)

    conor = dropdata.loc[ (dropdata['R__Name']  == name)]
    filtered_columns = conor.filter(regex='^B', axis=1)
    conorR = conor.drop(filtered_columns.columns, axis=1)

    new_columns = {col: col.replace("B__", "") for col in conorB.columns if col.startswith("B__")}
    conorB = conorB.rename(columns=new_columns)

    new_columns = {col: col.replace("R__", "") for col in conorR.columns if col.startswith("R__")}
    conorR = conorR.rename(columns=new_columns)

    combined_df = pd.concat([conorB, conorR], ignore_index=True)
    columns_to_calculate_mean = combined_df.columns.difference(['Name'])
    df_mean = combined_df[columns_to_calculate_mean].mean().to_frame().transpose()
    
    df_mean['Name'] = combined_df['Name']
    combined_all = pd.concat([combined_all, df_mean], ignore_index=True)

# Save the mean values to a new CSV file
combined_all.to_csv('fighter_means.csv', index=False, header=True)

In [17]:
#fighter_a = fighter_mean[fighter_mean["Name"] == "Alan Jouban"]
#fighter_b = fighter_mean[fighter_mean["Name"] == "Sam Alvey"]
#all_conc = pd.concat([df2.rename(columns=lambda x: "R__" + x)] + [df1.rename(columns=lambda x: "B__" + x)], axis=1)
fighter_mean

,Age,Height,Last_round,Max_round,Prev,Round1_Grappling_Reversals_Landed,Round1_Grappling_Standups_Landed,Round1_Grappling_Submissions_Attempts,Round1_Grappling_Takedowns_Attempts,Round1_Grappling_Takedowns_Landed,...,Round5_TIP_Ground Time,Round5_TIP_Guard Control Time,Round5_TIP_Half Guard Control Time,Round5_TIP_Misc. Ground Control Time,Round5_TIP_Mount Control Time,Round5_TIP_Neutral Time,Round5_TIP_Side Control Time,Round5_TIP_Standing Time,Streak,Weight
0,38.00,193.00,2.09,3.36,5.00,0.00,0.00,0.36,3.64,1.18,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.60,120.00
1,36.00,172.00,2.30,3.40,4.50,0.00,1.50,0.70,4.40,1.50,...,240.30,129.60,0.00,0.90,0.00,16.20,0.00,29.70,1.00,65.00
2,39.00,167.00,3.12,3.75,3.50,0.00,1.38,0.00,10.00,1.00,...,22.25,0.00,0.00,0.00,0.00,194.00,0.00,202.50,1.00,61.00
3,33.00,167.00,3.00,3.00,1.50,0.00,2.50,0.00,3.75,1.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,56.00
4,36.00,185.00,3.00,3.00,1.50,0.00,1.25,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.33,84.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,32.00,160.00,3.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,52.00
945,28.00,182.00,2.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,61.00
946,35.00,177.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,110.00
947,33.00,165.00,2.56,3.00,4.00,0.00,1.22,1.78,7.33,3.89,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,56.00


In [44]:
#pretvaranje object type columns  u kategorijske tip kolumni    
objecttypes = list(dropdata.select_dtypes(include=['O']).columns)
for col in objecttypes:
    dropdata[col] = dropdata[col].astype('category')

cat_columns = dropdata.select_dtypes(['category']).columns
dropdata[cat_columns+"_code"] = dropdata[cat_columns].apply(lambda x: x.cat.codes)

In [45]:
objecttypes = list(dropdata.select_dtypes(include=['float64']).columns)
for col in objecttypes:
    dropdata[col] = dropdata[col].astype('float32')
#dropdata = dropdata.astype(np.float32)

In [ ]:
dropdata.info()
dropdata = dropdata.drop(dropdata.columns[-2], axis=1)
#dropdata.loc[:,"winner_code"]
#dropdata.iloc[:,886]

In [48]:
#dropdata = data.drop(cat_columns,axis=1)
dropdata.to_csv('data_edited.csv', index=False)

In [ ]:
#for feature in data.dtypes[data.dtypes == 'object'].index: 
df = {'Name': data['B_Location'].value_counts().head(15).index, 'Count': data['B_Location'].value_counts().head(15).values}
df = pd.DataFrame(df)
sns.countplot(x='winby', data=data) 
plt.show() 

In [ ]:
df = {  'Name': data['B_Location'].value_counts().head(15).index, 
        'Count': data['B_Location'].value_counts().head(15).values}
df = pd.DataFrame(df)
sns.barplot(data=df, y='Name', x='Count')
plt.show() 

In [ ]:
for feature in data.dtypes[data.dtypes == 'object'].index: 
    sns.countplot(y=feature, data=dropdata) 
    plt.show() 

In [ ]:
sns.countplot(y='winner_code', data=dropdata) 
plt.show() 


In [ ]:
#for feature in data.dtypes[data.dtypes == 'object'].index: 
sns.countplot(y='R__Weight', data=dropdata) 
sns.countplot(y='B__Weight', data=dropdata) 
plt.show() 

In [ ]:
#Basic Correlation Matrix
corrmat = dropdata.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
# Subset Correlation Matrix
k = 10 #number of variables for heatmap
corrmat = dropdata.corr()
cols = corrmat.nlargest(k, 'winner')['winner'].index
cm = np.corrcoef(dropdata[cols].values.T)
sns.set(font_scale=1)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 5}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
Gradient Boosting Classifier was 
applied with a multinomial deviance loss 
function and a learning rate of 0.01, 
producing the following relative variable 
importances:

In [ ]:
sns.lmplot(x="B__Round3_Strikes_Body Significant Strikes_Attempts", 
            y="B__Round3_Strikes_Body Significant Strikes_Landed", 
            col="winner", hue="winner", data=dropdata, col_wrap=2)